In [1]:
import csv
import datetime
import matplotlib.pyplot as plt
import nltk
import joblib
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, cross_val_predict
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import plot_confusion_matrix, f1_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn import tree
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_validate
pd.set_option('display.max_rows', 10000)

In [2]:
def validate(date_text):
    try:
        datetime.datetime.strptime(date_text, '%Y-%m-%d')
    except ValueError:
        raise ValueError("Incorrect data format, should be YYYY-MM-DD")

In [3]:
italian_stemmer = SnowballStemmer('italian')
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([italian_stemmer.stem(w) for w in analyzer(doc)])

In [4]:
def search_tweet_Period(df, start_date = "2021-02-01", end_date = "2021-05-31"):
    
    validate(start_date)
    validate(end_date)
    
    return test_set[(test_set['timestamp'] > start_date) & (test_set['timestamp'] < end_date)]

In [16]:
#Load data and a short phase of preprocess
test_set = pd.read_csv("test_set.csv",sep=',')
test_set.drop_duplicates(subset=['tweet'])
#testset
#print(f"Number of tweets: {len(dataset)}")

,id,timestamp,username,tweet,likes_count,label
0,1365805791391985671,2021-02-28 00:27:30,filipporiccio1,l hanno gia data questa disponibilita moderna...,4,NaN
1,1365801638821306375,2021-02-28 00:11:00,cavinadenise,in molti che non sanno che non c era ancora u...,0,NaN
2,1365797552323518465,2021-02-27 23:54:46,biber0n1,il vaccino deve essere un farmaco generico pr...,1,NaN
3,1365794542172180481,2021-02-27 23:42:48,icolomboa,due mesi fa di questi giorni tra tanto scettic...,0,NaN
4,1365792763552428034,2021-02-27 23:35:44,attilaazurerive,tanta voglia di iri e di ritorno ai vecchi dis...,0,NaN
...,...,...,...,...,...,...
35291,1388313650797101057,2021-05-01 04:05:42,yy01329,il boom dei vaccini raggiunta quota 500 mila a...,2,NaN
35292,1388309589121187843,2021-05-01 03:49:34,claudiomarang1,del vaccino cubano ben che vada vedremo la lu...,1,NaN
35293,1388299190426644489,2021-05-01 03:08:15,bludichina,e il target primario dei vaccini attenuare i ...,0,NaN
35294,1388283826883944455,2021-05-01 02:07:12,ery975,canzone bella ma con una pecca nomina astraze...,1,NaN


In [17]:
news = []
with open('AccountToDelete.txt') as f:
    for line in f:
        news.append(line.strip())

In [18]:
indexNames = test_set[ test_set['username'].isin(news)].index
test_set.drop(indexNames, inplace=True)
#print(f"Number of tweets: {len(dataset)}")

In [25]:
test_set = test_set.sort_values(by=['likes_count'], ascending=False)

In [30]:
data = search_tweet_Period(test_set, "2021-02-01", "2021-02-28").copy()

In [31]:
data.to_csv("ConceptDriftFebbraio.csv")

In [ ]:
#Do classification by hand

In [ ]:
#Reload data
data = pd.read_csv("ConceptDrift-NomeMese-",sep=',',names=['tweet','label'])
training_set = pd.read_csv("training_set.csv",sep=',',names=['tweet','label'])

In [ ]:
#Merge the all the data
new_trainin_set = pd.concat(training_set,data)

In [ ]:
#Training the model
clf = Pipeline([
    ('vect', StemmedCountVectorizer(min_df=3, analyzer="word", stop_words = set(stopwords.words('italian')), ngram_range = (1,1))),
    ('tfidf', TfidfTransformer(smooth_idf=True,use_idf=True)),
    ('clf',BernoulliNB()),
])
clf.fit(new_trainin_set.tweet, new_trainin_set.label)

In [ ]:
#Save the new model
joblib.dump(clf, 'model-NomeMese-.pkl', compress=9)